datetime열에 주어지는 매 시점부터 총 14일(14×24 = 336시간) 분량의 적산차 변화를 예측

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

In [5]:
train = pd.read_csv("./data_city/data_tr_city.csv")
test = pd.read_csv("./data_city/data_ts_city.csv")
submit = pd.read_csv("./data_city/sample_city.csv")

In [6]:
train = train.fillna(method='bfill')

In [47]:
def ts_train_test(train, test, time_steps, for_periods): 
    """
    input:
     data: dataframe with dates and target data
    output:
     X_train, y_train: data from 2017/1/1 ~ 2020/12/31 
     X_test : data from 2021 - 
    time_steps: # of the input time steps 
    for_periods: # of the output time steps 
    """
    # create training and test set 
    ts_train = train.iloc[:,1].values.reshape(-1, 1)
    ts_test = test.iloc[:,1].values.reshape(-1, 1)
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)
    
    # create training data of s samples and t time steps 
    X_train = [] 
    y_train = [] 
    y_train_stacked = [] 
    for i in range(time_steps, ts_train_len - 1): 
        X_train.append(ts_train[i-time_steps:i,0])
        y_train.append(ts_train[i:i+for_periods,0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    # Reshapng X_train for efficient modelling 
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    
    # Preparing to creat X_test 
    inputs = ts_test
    inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
    inputs = inputs.reshape(-1,1)

    X_test = []
    for i in range(time_steps, ts_test_len+time_steps-for_periods):
        X_test.append(inputs[i-time_steps:i, 0])

    X_test = np.array(X_test).reshape(-1, 1)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, y_train, X_test 

X_train, y_train, X_test = ts_train_test(train, test, 672, 336)

print(X_train.shape[0], X_train.shape[1])
print(X_test.shape[0], X_test.shape[1])

34390 672
8088 1


C:\Users\kki96\AppData\Local\Temp\ipykernel_6412\4039959003.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train, y_train = np.array(X_train), np.array(y_train)
C:\Users\kki96\AppData\Local\Temp\ipykernel_6412\4039959003.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test).reshape(-1, 1)
